In [1]:
with open('EC2_pair.txt', 'r') as f:
    ppi_pairs = f.readlines() 


In [2]:
# !mkdir CV

In [4]:
with open('EC2_pair.txt', 'r') as f:
    ppi_pairs = f.readlines() 

for i in range(1):
    train_pairs_file = 'CV/train0'+'-'+str(i)
    test_pairs_file = 'CV/test0'+'-'+str(i)
    valid_pairs_file = 'CV/valid0'+'-'+str(i)
    with open(train_pairs_file, 'r') as f:
        train_pairs = f.readlines()
    with open(valid_pairs_file, 'r') as f:
        valid_pairs = f.readlines()
    train_valid_pairs = train_pairs + valid_pairs
    with open('CV/train_valid', 'w') as f:
        for line in train_valid_pairs:
            f.writelines(line)

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
for rep in range(3):
    n_splits = 10
    y_all_1 =  np.zeros((int(len(ppi_pairs)/2),1))
    y_all_2 =  np.ones( ( len(ppi_pairs) - int(len(ppi_pairs)/2)   ,1))
    y_all = np.concatenate((y_all_1, y_all_2), axis=0)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    skf_v = StratifiedKFold(n_splits=5, shuffle=True)

    for i, (train, test) in enumerate(skf.split(ppi_pairs, y_all)):
         
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(i)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(i)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(i)
        valid_len = int(train.shape[0]*0.1)
        train_valid_idx = np.arange(train.shape[0])
        np.random.shuffle(train_valid_idx)
         
         
        with open(train_pairs_file,'w') as f:
            for ind in train_valid_idx[:train.shape[0]-valid_len]:
                f.writelines(ppi_pairs[ind])
         
        with open(test_pairs_file,'w') as f:
            for ind in test:
                f.writelines(ppi_pairs[ind])
        
        with open(valid_pairs_file,'w') as f:
            for ind in train_valid_idx[train.shape[0]-valid_len:]:
                f.writelines(ppi_pairs[ind])

In [7]:
with open('sequence.txt', 'r') as f:
    all_seqs = f.readlines()

In [8]:
all_seqs[0]

'P0A6T9\tMSNVPAELKYSKEHEWLRKEADGTYTVGITEHAQELLGDMVFVDLPEVGATVSAGDDCAVAESVKAASDIYAPVSGEIVAVNDALSDSPELVNSEPYAGGWIFKIKASDESELESLLDATAYEALLEDE\n'

In [9]:
prot2seq = {}
for line in all_seqs:
    protID, seqs = line.rstrip().split('\t')
    prot2seq[protID] = seqs

In [12]:
prot2seq['P0A6T9']

'MSNVPAELKYSKEHEWLRKEADGTYTVGITEHAQELLGDMVFVDLPEVGATVSAGDDCAVAESVKAASDIYAPVSGEIVAVNDALSDSPELVNSEPYAGGWIFKIKASDESELESLLDATAYEALLEDE'

In [13]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [14]:
save_dict(prot2seq, 'ec2prot2seq.pkl')

In [15]:
with open('go_term.txt', 'r') as f:
    all_goterms = f.readlines()

In [16]:
all_goterms[0]

'P0A7U3\tGO:0003735,GO:0000049,GO:0005829,GO:0000028,GO:0022627,GO:0015935,GO:0006412,GO:0019843\n'

In [17]:
prot2go= {}
for line in all_goterms:
    protID, gos = line.rstrip().split('\t')
    prot2go[protID] = gos

In [18]:
prot2go['P0A7U3']

'GO:0003735,GO:0000049,GO:0005829,GO:0000028,GO:0022627,GO:0015935,GO:0006412,GO:0019843'

In [20]:
save_dict(prot2go, 'ec2prot2go.pkl')

In [21]:
max_golen = 0
for key, value in prot2go.items():
    gos = value.split(',')
    count = 0
    for go in gos:
        feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')
        count += feature.shape[0]-2
    if max_golen < count:
        max_golen = count
    

In [22]:
max_golen

157